# Book Recommendation System - Apriori Algorithm
---

## System Structure:
1. **Library Import**
2. **Data Loading**
3. **Data Preparation**
4. **Apriori Algorithm Application**
5. **Auxiliary Functions**
6. **Recommendation System**
7. **User Interface**
8. **Interactive System**

## System Features:
-  **Simple and Interactive**: Just type the book name
-  **5 Recommendations**: Always returns 5 suggestions
-  **Complete Titles**: Displays full book names
-  **Smart Search**: Finds books even with partial names
-  **Apriori Algorithm**: Based on purchase associations

---

# 0.0 - Library Import

In [51]:
import pandas as pd
import numpy as np
import warnings
import os
from collections import Counter
from kaggle.api.kaggle_api_extended import KaggleApi
from apyori import apriori

warnings.filterwarnings('ignore')
print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


# 1.0 - Data Loading

In [52]:
# Download dataset from Kaggle
api = KaggleApi()
api.authenticate()
os.makedirs("../dataset", exist_ok=True)
api.dataset_download_files('arashnic/book-recommendation-dataset', path='../dataset', unzip=True)

# Load data
df_books = pd.read_csv('../dataset/Books.csv')
df_ratings = pd.read_csv('../dataset/Ratings.csv')
df_users = pd.read_csv('../dataset/Users.csv')

print(f"📚 Books: {df_books.shape[0]:,} | ⭐ Ratings: {df_ratings.shape[0]:,} | 👥 Users: {df_users.shape[0]:,}")

Dataset URL: https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset
📚 Books: 271,360 | ⭐ Ratings: 1,149,780 | 👥 Users: 278,858
📚 Books: 271,360 | ⭐ Ratings: 1,149,780 | 👥 Users: 278,858


# 2.0 - Data Preparation

In [53]:
# Prepare data for Apriori
df_ratings_high = df_ratings[df_ratings['Book-Rating'] >= 7].copy()
user_counts = df_ratings_high['User-ID'].value_counts()
active_users = user_counts[user_counts >= 3].index
df_apriori = df_ratings_high[df_ratings_high['User-ID'].isin(active_users)].copy()

# Create transactions
def create_transactions(df):
    """Creates transactions for the Apriori algorithm"""
    transactions = []
    for user_id in df['User-ID'].unique():
        user_books = df[df['User-ID'] == user_id]['ISBN'].tolist()
        if len(user_books) >= 2:
            transactions.append(user_books)
    return transactions

transactions = create_transactions(df_apriori)
all_books = [item for sublist in transactions for item in sublist]
book_freq = Counter(all_books)

print(f"📦 Transactions: {len(transactions):,} | 📚 Unique books: {len(book_freq):,}")

📦 Transactions: 17,491 | 📚 Unique books: 129,511


# 3.0 - Apriori Algorithm Application

In [54]:
# Apply Apriori algorithm
association_rules = apriori(
    transactions=transactions,
    min_support=0.001,
    min_confidence=0.1,
    min_lift=1.0,
    min_length=2,
    max_length=2
)

rules_list = list(association_rules)
print(f"📋 Apriori rules: {len(rules_list)}")

# If no rules found, use co-occurrence
if len(rules_list) == 0:
    popular_books = book_freq.most_common(15)
    rules_list = []
    
    for i, (book1, count1) in enumerate(popular_books[:10]):
        for j, (book2, count2) in enumerate(popular_books[:10]):
            if i != j:
                co_occurrence = sum(1 for t in transactions if book1 in t and book2 in t)
                if co_occurrence > 1:
                    support = co_occurrence / len(transactions)
                    confidence = co_occurrence / count1
                    lift = confidence / (count2 / len(transactions))
                    
                    rules_list.append({
                        'antecedent': book1,
                        'consequent': book2,
                        'support': support,
                        'confidence': confidence,
                        'lift': lift
                    })
    
    rules_list.sort(key=lambda x: x['lift'], reverse=True)
    print(f"📋 Co-occurrence rules: {len(rules_list)}")

📋 Apriori rules: 143


# 4.0 - Auxiliary Functions

In [55]:
# Function to get book information
def get_book_info(book_id):
    """Gets detailed book information by ISBN"""
    try:
        book_data = df_books[df_books['ISBN'] == book_id]
        if len(book_data) > 0:
            book = book_data.iloc[0]
            return {
                'title': book['Book-Title'],
                'author': book['Book-Author'],
                'year': book['Year-Of-Publication']
            }
    except:
        pass
    return {'title': f'Book {book_id}', 'author': 'Unknown author', 'year': 'N/A'}

# Function to search books
def search_books_by_title(title_search, max_results=10):
    """Searches books by title"""
    title_search = title_search.lower()
    books_found = df_books[
        df_books['Book-Title'].str.lower().str.contains(title_search, na=False)
    ]
    
    if books_found.empty:
        return []
    
    results = []
    for idx, book in books_found.head(max_results).iterrows():
        results.append({
            'isbn': book['ISBN'],
            'title': book['Book-Title'],
            'author': book['Book-Author'],
            'year': book['Year-Of-Publication']
        })
    return results

print("✅ Auxiliary functions created")

✅ Auxiliary functions created


# 5.0 - Main Recommendation System

In [56]:
def recommend_books(user_books, n_recommendations=5):
    """Main recommendation system based on Apriori algorithm"""
    recommendations = {}
    
    # Try to use Apriori rules first
    if len(rules_list) > 0 and hasattr(rules_list[0], 'ordered_statistics'):
        for user_book in user_books:
            for rule in rules_list:
                antecedent = list(rule.ordered_statistics[0].items_base)
                consequent = list(rule.ordered_statistics[0].items_add)
                
                if user_book in antecedent:
                    for rec_book in consequent:
                        if rec_book not in user_books:
                            score = (rule.ordered_statistics[0].confidence * 
                                   rule.ordered_statistics[0].lift)
                            recommendations[rec_book] = recommendations.get(rec_book, 0) + score
    
    # Use co-occurrence rules
    elif len(rules_list) > 0 and isinstance(rules_list[0], dict):
        for user_book in user_books:
            for rule in rules_list:
                if rule['antecedent'] == user_book:
                    rec_book = rule['consequent']
                    if rec_book not in user_books:
                        score = rule['confidence'] * rule['lift']
                        recommendations[rec_book] = recommendations.get(rec_book, 0) + score
    
    # Fallback by author
    if not recommendations:
        user_authors = set()
        for user_book in user_books:
            book_info = get_book_info(user_book)
            if book_info['author'] != 'Unknown author':
                user_authors.add(book_info['author'])
        
        for author in user_authors:
            author_books = df_books[df_books['Book-Author'] == author]['ISBN'].tolist()
            for isbn in author_books:
                if isbn not in user_books and isbn in book_freq:
                    score = book_freq[isbn] * 0.5
                    recommendations[isbn] = recommendations.get(isbn, 0) + score
    
    # Fallback by popularity
    if not recommendations:
        popular_books = book_freq.most_common(20)
        for isbn, count in popular_books:
            if isbn not in user_books:
                recommendations[isbn] = count * 0.1
                if len(recommendations) >= n_recommendations:
                    break
    
    # Sort and format result
    sorted_recs = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
    
    result = []
    for isbn, score in sorted_recs[:n_recommendations]:
        book_info = get_book_info(isbn)
        result.append({
            'isbn': isbn,
            'title': book_info['title'],
            'author': book_info['author'],
            'score': score
        })
    
    return result

print("✅ Recommendation system created")

✅ Recommendation system created


# 6.0 - User Interface

In [57]:
# Function to display recommendations
def display_recommendations(book_input, recommendations):
    """Displays formatted recommendations"""
    print(f"\n📚 RECOMMENDATIONS FOR: {book_input['title']}")
    print(f"👤 Author: {book_input['author']}")
    print("=" * 60)
    
    if not recommendations:
        print("❌ No recommendations found")
        return
    
    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec['title'][:50]}...")
        print(f"   👤 {rec['author'][:30]}...")
        print(f"   📊 Score: {rec['score']:.2f}\n")

# Simplified interactive system
def search_and_recommend(title_search, n_recommendations=5):
    """Main function to search and recommend books"""
    print(f"🔍 Searching for '{title_search}'...")
    
    # Search books
    books_found = search_books_by_title(title_search, 5)
    
    if not books_found:
        print(f"❌ No books found with '{title_search}'")
        return
    
    # Use the first book found
    selected_book = books_found[0]
    print(f"✅ Found: {selected_book['title']}")
    
    # Generate recommendations
    recommendations = recommend_books([selected_book['isbn']], n_recommendations)
    
    # Display result
    display_recommendations(selected_book, recommendations)
    
    return recommendations

print("✅ Interface functions created")

✅ Interface functions created


# 7.0 - Interactive System

In [58]:
def interactive_system():
    """Interactive system where you type the book name and get recommendations"""
    print("="*80)
    print("🎉 INTERACTIVE BOOK RECOMMENDATION SYSTEM")
    print("="*80)
    print("💡 Type the book name to get 5 recommendations")
    print("💡 Type 'exit' to close the system")
    print("="*80)
    
    while True:
        # Request book name
        title_search = input("\n🔍 Enter book name: ").strip()
        
        # Check if wants to exit
        if title_search.lower() in ['exit', 'quit', 'sair', '']:
            print("\n👋 Thank you for using the recommendation system!")
            print("🎯 We hope we helped you discover new books!")
            break
        
        print(f"\n🔍 Searching for '{title_search}'...")
        
        # Search books
        books_found = search_books_by_title(title_search, max_results=1)
        
        if not books_found:
            print(f"❌ No books found with '{title_search}'")
            print("💡 Try using different keywords")
            continue
        
        # Use the first book found
        selected_book = books_found[0]
        print(f"✅ Book found: {selected_book['title']}")
        print(f"👤 Author: {selected_book['author']}")
        
        print("\n🤖 Generating 5 recommendations...")
        
        # Generate recommendations
        recommendations = recommend_books([selected_book['isbn']], n_recommendations=5)
        
        # Display recommendations with complete titles
        print(f"\n📚 RECOMMENDATIONS FOR: {selected_book['title']}")
        print(f"👤 Author: {selected_book['author']}")
        print("="*80)
        
        if recommendations:
            for i, rec in enumerate(recommendations, 1):
                print(f"{i}. 📖 {rec['title']}")
                print(f"   👤 {rec['author']}")
                print(f"   📊 Score: {rec['score']:.2f}\n")
        else:
            print("❌ No recommendations found")
        
        print(f"✅ Total recommendations: {len(recommendations)}")
        print("="*80)

# 🚀 START INTERACTIVE SYSTEM
print("🚀 To use the interactive system, run: interactive_system()")
print("💡 Or run the cell below to start automatically")

🚀 To use the interactive system, run: interactive_system()
💡 Or run the cell below to start automatically


# 8.0 - Interactive System Execution

In [59]:
def interactive_system():
    """Interactive system for book recommendation"""
    print("🎯 Book Recommendation System - Ready to use!")
    print("=" * 50)
    
    while True:
        print("\n📖 Enter the name of the book you liked:")
        print("   (or 'exit' to quit)")
        
        book_name = input("➤ ").strip()
        
        if book_name.lower() in ['exit', 'quit', 'sair', '']:
            print("👋 Thank you for using the system! See you later!")
            break
        
        print(f"\n🔍 Searching for recommendations for: '{book_name}'")
        print("-" * 40)
        
        # Search and recommend (this function already calls display_recommendations internally)
        recommendations = search_and_recommend(book_name, n_recommendations=5)
        
        if not recommendations:
            print(f"❌ We couldn't find the book '{book_name}' in our database.")
            print("💡 Try another name or check the spelling.")
        
        print("\n" + "="*50)

# 🚀 START INTERACTIVE SYSTEM
print("🚀 System configured successfully!")
print("💡 To start the interactive system, run: interactive_system()")
print("💡 For simplified version, run: interactive_system_simple()")

🚀 System configured successfully!
💡 To start the interactive system, run: interactive_system()
💡 For simplified version, run: interactive_system_simple()


In [60]:
interactive_system()

🎯 Book Recommendation System - Ready to use!

📖 Enter the name of the book you liked:
   (or 'exit' to quit)

🔍 Searching for recommendations for: 'the alchemist'
----------------------------------------
🔍 Searching for 'the alchemist'...
✅ Found: The Alchemist: A Fable About Following Your Dream

📚 RECOMMENDATIONS FOR: The Alchemist: A Fable About Following Your Dream
👤 Author: Paulo Coelho
1. The Alchemist: A Fable About Following Your Dream...
   👤 Paulo Coelho...
   📊 Score: 27.50

2. Veronika Decides to Die...
   👤 Paulo Coelho...
   📊 Score: 7.50

3. Der Alchimist....
   👤 Paulo Coelho...
   📊 Score: 4.00

4. Veronika Decides to Die...
   👤 Paulo Coelho...
   📊 Score: 4.00

5. L'Alchimiste...
   👤 Paulo Coelho...
   📊 Score: 3.00



📖 Enter the name of the book you liked:
   (or 'exit' to quit)

🔍 Searching for recommendations for: 'the alchemist'
----------------------------------------
🔍 Searching for 'the alchemist'...
✅ Found: The Alchemist: A Fable About Following Your Dream